# INDIVIDUAL MOBILITY NETWORK

### IMPORT LIBRARIES

In [ ]:
import numpy as np
import folium
import networkx as nx
import json
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph
import pickle
from matplotlib import cm as cm
import pydot 
from networkx.drawing.nx_agraph import write_dot
import pandas as pd

%matplotlib inline

#### Define the parameters to select the correct area and time period

In [ ]:
stop = '5'
id_area = '2'
month =  '9'
n_months = '2'
week = '0'

month_code = month
if n_months != "1":
    for m in range(1, int(n_months)):
        month_code += "_" + str(int(month)+m)

In [ ]:
vehicles = ['7410_74050', '15000_131630', '11770_93490'] ## area 2
# vehicles = ['14550_128520', '8240_96240', '800_13540'] ## area 11

#### Open the dataframe of the location features

In [ ]:
path = '../../../datasets/out/Traj'+stop+'min/'
file_name_in = 'imn_light_area'+id_area+'_month'+month_code+'_week'+ week
file_name_out = '_area'+id_area+'_month'+month_code+'_week'+ week

#### Function to load the light imn

In [ ]:
def open_file(path_file):
    with open(path_file+'.json', 'r') as f:
        file_j = json.load(f)
        return file_j

#### Transform the json graph into a networkx friendly

In [ ]:
def from_imn_to_graph(g):
    return json_graph.node_link_graph(g, attrs={'link': 'edges', 'source': 'from', 'target': 'to'})

#### Translate a value from a range (min_w, max_w) to a range (a, b)

In [ ]:
def translate(value, min_w, max_w, a, b):
    # Figure out how 'wide' each range is
    diff = max_w - min_w
    span = b - a

    if diff != 0:
        # Convert the left range into a 0-1 range (float)
        v_scaled = float(value - min_w) / float(diff)

        # Convert the 0-1 range into a value in the right range.
        return a + (v_scaled * span)
    else:
        return value

#### Scale the egdes weight in the range (0.3, 9.0)

In [ ]:
def scale_weights(edges):
    w = []
    for e in edges:
        w.append(e["weight"])

    min_w = min(w)
    max_w = max(w)
    r = float(max_w - min_w)
    return [translate(wi, min_w, max_w, 0.3, 10) for wi in w]

#### Extract the nodes size (using the support of the location)

In [ ]:
def nodes_sizes(imn):
    nodes_s = []
    
    loc_feat = imn["location_features"]
    locs = list(loc_feat.keys())
    for l in locs:
        loc_info = loc_feat[str(l)]
        if "loc_support" in loc_info:
            nodes_s.append(loc_info["loc_support"])
        else:
            nodes_s.append(0)
    return nodes_s

#### Scale the nodes size in the range (400, 1500)

In [ ]:
def scale_nodes(nodes):
    min_n = min(nodes)
    max_n = max(nodes)
    r = float(max_n - min_n)
    return [translate(ni, min_n, max_n, 500, 2000) for ni in nodes]

#### Draw the imn using networkx

In [ ]:
def draw_imn(v, G, nodes_s, w_scaled, file_name_in):
    options = {
        'node_color': 'black',
        'font_color': "white",
        'font_size': 13,
        'arrows': False,
        'font_weight': "bold"
    }
    fig = plt.figure(figsize=(20, 20)) 

    nx.draw_circular(G, with_labels=True, node_size=nodes_s, width=w_scaled, **options)
    plt.savefig('../../../thesis/images/'+file_name_in+'_'+v+'.png', format='png', bbox_inches='tight')
    plt.close(fig)

#### For each vehicle compute the imn and store the result

In [ ]:
def store_imns(path, file_name_in):
    f = open_file(path+file_name_in)
    print("There are", len(list(f)), "vehicles.")
    
    for v in vehicles:
        g = f[v]["graph"]
        G = from_imn_to_graph(g)
        
        w_scaled = scale_weights(g["edges"])
        
        nodes_order = G.nodes
        nodes_weights = scale_nodes(nodes_sizes(f[v]))

        nodes_ordered = []
        for pos in nodes_order:
            nodes_ordered.append(nodes_weights[pos])
        
        draw_imn(v, G, nodes_ordered, w_scaled, file_name_in)

#### Compute some statistics on the imns

In [ ]:
def imn_stats(path, file_name_in):
    f = open_file(path+file_name_in)
    
    n_nodes = []
    n_edges = []
    densities = []
    clust_coeff = []
    degree_centrality = []
    avg_node_degree = []
    
    for v in list(f):
        g = f[v]["graph"]
        G = from_imn_to_graph(g)
        
        n_nodes.append(len(G.nodes))
        n_edges.append(len(G.edges))
        densities.append(nx.density(G))
        clust_coeff.append(nx.average_clustering(G))
        degree_centrality.append(np.mean(list(nx.degree_centrality(G).values())))
        degree_list = [d[1] for d in G.degree]
        avg_node_degree.append(np.mean(degree_list))
        
    print("mean(n_nodes)", np.mean(n_nodes))
    print("std(n_nodes)", np.std(n_nodes))
    print("mean(n_edges)", np.mean(n_edges))
    print("std(n_edges)", np.std(n_edges))
    print("mean(densities)", np.mean(densities))
    print("std(densities)", np.std(densities))
    print("mean(clust_coeff)", np.mean(clust_coeff))
    print("std(clust_coeff)", np.std(clust_coeff))
    print("mean(degree_centrality)", np.mean(degree_centrality))
    print("std(degree_centrality)", np.std(degree_centrality))
    print("mean(avg_node_degree)", np.mean(avg_node_degree))
    print("std(avg_node_degree)", np.std(avg_node_degree))

In [ ]:
store_imns(path, file_name_in)

In [ ]:
imn_stats(path, file_name_in)

In [ ]:
# ######### TO DRAW SINGLE IMN WITH WHITE NODES

# f = open_file(path+file_name_in)
# v = "16020_139450"
# g = f[v]["graph"]
# G = from_imn_to_graph(g)

# w_scaled = scale_weights(g["edges"])

# nodes_order = G.nodes
# nodes_weights = scale_nodes(nodes_sizes(f[v]))

# nodes_ordered = []
# for pos in nodes_order:
#     nodes_ordered.append(nodes_weights[pos])

# draw_imn(v, G, nodes_ordered, w_scaled, file_name_in+"JJJ")

# SEMANTIC INDIVIDUAL MOBILITY NETWORK

#### Define a new function to draw the sem imn, so the same graphs but with the nodes colored according to the cluster

In [ ]:
def draw_sem_imn(v, G, color_nodes, nodes_s, w_scaled, file_name_in):
    options = {
        'node_color': color_nodes,
        'font_color': "black",
        'font_size': 13,
        'arrows': False,
        'font_weight': "bold"
    }
    fig = plt.figure(figsize=(20, 20)) 

    nx.draw_circular(G, with_labels=True, node_size=nodes_s, width=w_scaled, **options)
    plt.savefig('../../../thesis/images/sem_imn_'+file_name_out+'_'+v+'.png', format='png', bbox_inches='tight')
    plt.close(fig)

#### Read the list of cluster ids

In [ ]:
file_name = '_area'+id_area+'_month'+month_code+'_week'+ week

with open(path + "link_cluster" + file_name + '_log.pickle', 'rb') as fp:
    df_link = pickle.load(fp)
    link_cluster = df_link["link_cluster"]

#### For each vehicle extract the list of cluster id, compute the sem imn and store the result

In [ ]:
def store_sem_imns(path, file_name_in):
    f = open_file(path+file_name_in)
    for v in vehicles:
        g = f[v]["graph"]
        G = from_imn_to_graph(g)
        w_scaled = scale_weights(g["edges"])
                
        df_link_v = df_link[df_link["vehicle"] == v]        
        colors = ["#525252", "#ffff33", "#e31a1c", "#33a02c", "#1f78b4", "#e7298a"]
        color_nodes = [colors[c-1] for c in list(df_link_v["link_cluster"])]
        
        #print(list(df_link_v["link_cluster"]))
        print(len(list(df_link_v["link_cluster"])))
        nodes_order = G.nodes
        nodes_weights = scale_nodes(nodes_sizes(f[v]))
        
        print(len(nodes_order))
        print(len(color_nodes))

        colors_ordered = []
        nodes_ordered = [] 
        for pos in nodes_order:
            nodes_ordered.append(nodes_weights[pos])
            colors_ordered.append(color_nodes[pos])
            
        print(len(colors_ordered))
        print(np.sort(G.nodes))
        
        draw_sem_imn(v, G, colors_ordered, nodes_ordered, w_scaled, file_name_in)

In [ ]:
##### NEW VERSION

file_name = '_area'+id_area+'_month'+month_code+'_week'+ week
file_name_in = 'loc_feat'+ file_name + '_complete.csv'

# read dataframe with location features, extract only the ones we care about
df_complete = pd.read_csv(path+file_name_in)
df_complete = df_complete[["vehicle", "loc_id", "loc_proto_lat", "loc_proto_lon", "support"]]

# read the list of linkage clusters for each location
with open(path + "link_cluster" + file_name + '_log.pickle', 'rb') as fp:
    df_link = pickle.load(fp)
    link_cluster = df_link["link_cluster"]

# store the linkage cluster as new column of the df
df_complete = df_complete.assign(link_cluster = link_cluster) 



def store_sem_imns(path, file_name_in):
    
    with open(path+"imn_light"+file_name_out+'.json', 'rb') as fp:
        f = json.load(fp)
        for v in vehicles:
            
            location_nextlocs = f[v]["location_nextlocs"]
            
            g = f[v]["graph"]
            G = from_imn_to_graph(g)
            
            print(len(G.nodes)) #1
            print(len(G.edges)) #2
            w_scaled = scale_weights(g["edges"])

            df_link_v = df_link[df_link["vehicle"] == v]        
            colors = ["#525252", "#ffff33", "#e31a1c", "#33a02c", "#1f78b4", "#e7298a"]
            color_nodes = [colors[c-1] for c in list(df_link_v["link_cluster"])]
            
            G = nx.DiGraph()
            G.add_nodes_from(range(len(df_link_v)))
            
            for lid1 in location_nextlocs:
                for lid2 in location_nextlocs[lid1]:
                    G.add_edge(lid1, lid2, weight=location_nextlocs[lid1][lid2])
                    
            print(len(G.nodes)) #3
            print(len(G.edges)) #4
            
            print(list(df_link_v["link_cluster"]))
            print(len(list(df_link_v["link_cluster"])))
            nodes_order = G.nodes
            nodes_weights = scale_nodes(nodes_sizes(f[v]))

            print(len(nodes_order))
            print(len(color_nodes))

            colors_ordered = []
            nodes_ordered = [] 
            for pos in nodes_order:
                nodes_ordered.append(nodes_weights[pos])
                colors_ordered.append(color_nodes[pos])

            print(len(colors_ordered))
            print(np.sort(G.nodes))

            draw_sem_imn(v, G, colors_ordered, nodes_ordered, w_scaled, file_name_in)

In [ ]:
def visualize_imn(vis_traj = True, tiles='stamentoner', cluster_on=True):
    # get dataframe not normalized
    file_name = '_area'+id_area+'_month'+month_code+'_week'+ week
    file_name_in = 'loc_feat'+ file_name + '_complete.csv'

    # read dataframe with location features, extract only the ones we care about
    df_complete = pd.read_csv(path+file_name_in)
    df_complete = df_complete[["vehicle", "loc_id", "loc_proto_lat", "loc_proto_lon", "support"]]

    # read the list of linkage clusters for each location
    with open(path + "link_cluster" + file_name + '_log.pickle', 'rb') as fp:
        df_link = pickle.load(fp)
        link_cluster = df_link["link_cluster"]

    # store the linkage cluster as new column of the df
    df_complete = df_complete.assign(link_cluster = link_cluster) 

    # extract 3 vehicle with different purity levels
    vehicles = ['7410_74050', '15000_131630', '11770_93490']

    for ix, v in enumerate(vehicles):

        # extract the df of locations only of that vehicle
        df_v = df_complete[df_complete["vehicle"] == v]
        
        # read the imn of the vehicle
        with open(path+"imn_light"+file_name+'.json', 'rb') as fp:
            file_j = json.load(fp)
            imn_v = file_j[v]
        
        # extract the dict of nextlocations and the location features
        location_nextlocs = imn_v["location_nextlocs"]

        # compute a list of points coordinates
        points = np.array([[p[0], p[1]] for p in zip(df_v["loc_proto_lon"], df_v["loc_proto_lat"])])

        # compute a dict from the location id (as string) to its coordinates
        loc_id_string = [str(x) for x in df_v["loc_id"]]
        location_prototype = dict(zip(loc_id_string, points))

        lat_list = list(df_v["loc_proto_lat"])
        lon_list = list(df_v["loc_proto_lon"])
        sup_list = list(df_v["support"])

        # nero, giallo, rosso, verde, blu, viola
        colors = ["#525252", "#ffff33", "#e31a1c", "#33a02c", "#1f78b4", "#e7298a"]
        link_cluster = df_v["link_cluster"]
        #print(list(link_cluster))
        sup_colors = [colors[l-1] for l in link_cluster]
        
        G = nx.DiGraph()
        G.add_nodes_from(range(0, len(lon_list)))
        
        w = []

        for lid1 in location_nextlocs:
            for lid2 in location_nextlocs[lid1]:
                w.append(location_nextlocs[lid1][lid2])
                G.add_edge(lid1, lid2, weight=location_nextlocs[lid1][lid2])
                
        min_w = min(w)
        max_w = max(w)
        r = float(max_w - min_w)
        w_scaled = [translate(wi, min_w, max_w, 0.3, 10) for wi in w]


        for i in range(0, len(lon_list)):
            print(i, sup_list[i], sup_colors[i])
            
        draw_sem_imn(v, G, sup_colors, sup_list, w_scaled, 'imn_light_area'+id_area+'_month'+month_code+'_week'+ week)

    return 0
visualize_imn()

In [ ]:
store_sem_imns(path, file_name_in)

# CLUSTERED SEMANTIC INDIVIDUAL MOBILITY NETWORK

#### Trasform a color from rgba to hex

In [ ]:
def from_rgba_to_hex(a, b, c):
    a = int(a*255)
    b = int(b*255)
    c = int(c*255)
    return '#%02x%02x%02x' % (a, b, c)

In [ ]:
def draw_pydot_graph(node_weights, colors, nodes_id, edge_weight, file_name_in, v):
    graph = pydot.Dot(graph_type='graph')

    graph.set_node_defaults(shape= "circle", fixedsize = True, style='filled', color='white')
    graph.set_edge_defaults(color="black", dir="forward", arrowhead="vee", arrowsize="0.5", weight="50", minlen=10, mindist=3)

    pydot_node_weights = np.divide(node_weights, 5000)

    nodes =[]
    for i in range(len(nodes_id)):
        nodes.append(pydot.Node(str(nodes_id[i]), fillcolor=colors[i], width=str(pydot_node_weights[i])))
        graph.add_node(nodes[i])
        
    from_node_id_to_pos = dict(zip(nodes_id, range(len(nodes_id))))

    for e, w in edge_weight.items():
        n1_pos = from_node_id_to_pos[e[0]]
        n2_pos = from_node_id_to_pos[e[1]]
        graph.add_edge(pydot.Edge(nodes[n1_pos], nodes[n2_pos], penwidth=str(w/3), mindist=10))

    graph.write_png('../../../thesis/images/clust_sem_imn_'+file_name_out+'_'+v+'.png', prog='circo')

In [ ]:
def draw_annotated_imn(G, df_link_v, v):
    # compute the new list of nodes weights
    node_weights = []
    # compute the new list of nodes colors
    colors = []
    # a dict from location id to cluster id
    from_loc_to_cluster = dict()
    nodes_id = []

    # for each cluster take the corresponding df
    for i in range(1,7):
        df_i = df_link_v[df_link_v["link_cluster"] == i]
        # if there's at least a location of that cluster
        if len(df_i) != 0:
            nodes_id.append(i)
            # insert all location in that dict
            for i, n in enumerate(df_i["loc_id"]):
                from_loc_to_cluster[n] = list(df_i["link_cluster"])[i]
                
    node_weights = scale_nodes(node_weights)
    
    # transform the old edges into the new ones
    new_edges = []
    edge_weight = dict()
    
    for ed in G.edges.data('weight'):
        new_edges.append((from_loc_to_cluster[ed[0]], from_loc_to_cluster[ed[1]]))
        first_node = from_loc_to_cluster[ed[0]]
        second_node = from_loc_to_cluster[ed[1]]
        edge_weight[from_loc_to_cluster[ed[0]], from_loc_to_cluster[ed[1]]] = 0

    for ed in G.edges.data('weight'):
        new_edges.append((from_loc_to_cluster[ed[0]], from_loc_to_cluster[ed[1]]))
        first_node = from_loc_to_cluster[ed[0]]
        second_node = from_loc_to_cluster[ed[1]]
        edge_weight[from_loc_to_cluster[ed[0]], from_loc_to_cluster[ed[1]]] += ed[2]
            
    w = edge_weight.values()
    
    min_w = min(w)
    max_w = max(w)
    r = float(max_w - min_w)
    edge_scaled = [translate(wi, min_w, max_w, 0.3, 10) for wi in w]
        
    edge_weight = dict(zip(edge_weight.keys(), edge_scaled))
        
    #draw_pydot_graph(node_weights, colors, nodes_id, edge_weight, file_name_in, v)
    
    return nodes_id, edge_weight.keys()

#### For each vehicle collapse the nodes according to the clustering, compute the annotated imn and store the result

In [ ]:
def store_annotated_imns(path, file_name_in):
    nodes_list = []
    edge_list = []
    
    f = open_file(path+file_name_in)
    for v in list(f): #vehicles:
        # compute the imn graph
        g = f[v]["graph"]
        G = from_imn_to_graph(g)
        
        df_link_v = df_link[df_link["vehicle"] == v]        
        
        nodes_order = G.nodes
        nodes_weights = scale_nodes(nodes_sizes(f[v]))

        nodes_ordered = [] 
        for pos in nodes_order:
            nodes_ordered.append(nodes_weights[pos])
            
        print(len(df_link_v["link_cluster"]))
        print(len(G.nodes))

        # take only the data about the vehicle
        
        df_link_v = df_link_v.assign(graph_node = nodes_ordered) 

        nodes_id, edges = draw_annotated_imn(G, df_link_v, v)
        nodes_list.append(nodes_id)
        edge_list.append(edges)
        
    return nodes_list, edge_list

In [ ]:
def scale_nodes(nodes):
    min_n = min(nodes)
    max_n = max(nodes)
    r = float(max_n - min_n)
    return [translate(ni, min_n, max_n, 1000, 2500) for ni in nodes]

In [ ]:
nodes_list, edge_list = store_annotated_imns(path, file_name_in)

In [ ]:
nodes_list = []
edge_list = []

# get dataframe not normalized
file_name = '_area'+id_area+'_month'+month_code+'_week'+ week
file_name_in = 'loc_feat'+ file_name + '_complete.csv'

# read dataframe with location features, extract only the ones we care about
df_complete = pd.read_csv(path+file_name_in)
df_complete = df_complete[["vehicle", "loc_id", "loc_proto_lat", "loc_proto_lon"]]

# read the list of linkage clusters for each location
with open(path + "link_cluster" + file_name + '_log.pickle', 'rb') as fp:
    df_link = pickle.load(fp)
    link_cluster = df_link["link_cluster"]
    
# read the imn of the vehicle
with open(path+"imn_light"+file_name+'.json', 'rb') as fp:
    file_j = json.load(fp)

# store the linkage cluster as new column of the df
df_complete = df_complete.assign(link_cluster = link_cluster) 

f = open_file(path+'imn_light_area'+id_area+'_month'+month_code+'_week'+ week)
for v in list(f):    
    g = f[v]["graph"]
    G1 = from_imn_to_graph(g)

    # extract the df of locations only of that vehicle
    df_v = df_complete[df_complete["vehicle"] == v]

    imn_v = file_j[v]

    # extract the dict of nextlocations and the location features
    location_nextlocs = imn_v["location_nextlocs"]

    lat_list = list(df_v["loc_proto_lat"])
    lon_list = list(df_v["loc_proto_lon"])

    link_cluster = df_v["link_cluster"]

    G = nx.Graph()
    G.add_nodes_from(G1.nodes)

    for lid1 in location_nextlocs:
        for lid2 in location_nextlocs[lid1]:
            w.append(location_nextlocs[lid1][lid2])
            G.add_edge(int(lid1), int(lid2))
                    
    # a dict from location id to cluster id
    from_loc_to_cluster = dict()
    nodes_id = []

    # for each cluster take the corresponding df
    for i in range(1,7):
        df_i = df_v[df_v["link_cluster"] == i]
        # if there's at least a location of that cluster
        if len(df_i) != 0:
            nodes_id.append(i)
            # insert all location in that dict
            for i, n in enumerate(df_i["loc_id"]):
                from_loc_to_cluster[n] = list(df_i["link_cluster"])[i]
                            
    # transform the old edges into the new ones
    new_edges = []
    
    for ed in G.edges:
        new_edges.append((from_loc_to_cluster[ed[0]], from_loc_to_cluster[ed[1]]))
    
    nodes_list.append(nodes_id)
    edge_list.append(new_edges)

In [ ]:
n_nodes = [] #
n_edges = []
densities = []
clust_coeff = []
degree_centrality = []
avg_node_degree = []

for i, v in enumerate(list(f)):
    G = nx.DiGraph()
    G.add_nodes_from(nodes_list[i])
    G.add_edges_from(edge_list[i])
    G = G.to_undirected()

    n_nodes.append(len(G.nodes))
    n_edges.append(len(G.edges))
    densities.append(nx.density(G))
    clust_coeff.append(nx.average_clustering(G))
    degree_centrality.append(np.mean(list(nx.degree_centrality(G).values())))
    degree_list = [d[1] for d in G.degree]
    avg_node_degree.append(np.mean(degree_list))

print("mean(n_nodes)", np.mean(n_nodes))
print("std(n_nodes)", np.std(n_nodes))
print("mean(n_edges)", np.mean(n_edges))
print("std(n_edges)", np.std(n_edges))
print("mean(densities)", np.mean(densities))
print("std(densities)", np.std(densities))
print("mean(avg_node_degree)", np.mean(avg_node_degree))
print("std(avg_node_degree)", np.std(avg_node_degree))
print("mean(clust_coeff)", np.mean(clust_coeff))
print("std(clust_coeff)", np.std(clust_coeff))
print("mean(degree_centrality)", np.mean(degree_centrality))
print("std(degree_centrality)", np.std(degree_centrality))

In [ ]:
def imn_annotated_stats(path, file_name_in, nodes_list, edge_list):
    
    n_nodes = [] #
    n_edges = []
    densities = []
    clust_coeff = []
    degree_centrality = []
    avg_node_degree = []
    
    for i, v in enumerate(list(f)):
        G = nx.DiGraph()
        G.add_nodes_from(nodes_list[i])
        G.add_edges_from(edge_list[i])
        G = G.to_undirected()

        n_nodes.append(len(G.nodes))
        n_edges.append(len(G.edges))
        densities.append(nx.density(G))
        clust_coeff.append(nx.average_clustering(G))
        degree_centrality.append(np.mean(list(nx.degree_centrality(G).values())))
        degree_list = [d[1] for d in G.degree]
        avg_node_degree.append(np.mean(degree_list))
        
    print("mean(n_nodes)", np.mean(n_nodes))
    print("std(n_nodes)", np.std(n_nodes))
    print("mean(n_edges)", np.mean(n_edges))
    print("std(n_edges)", np.std(n_edges))
    print("mean(densities)", np.mean(densities))
    print("std(densities)", np.std(densities))
    print("mean(avg_node_degree)", np.mean(avg_node_degree))
    print("std(avg_node_degree)", np.std(avg_node_degree))
    print("mean(clust_coeff)", np.mean(clust_coeff))
    print("std(clust_coeff)", np.std(clust_coeff))
    print("mean(degree_centrality)", np.mean(degree_centrality))
    print("std(degree_centrality)", np.std(degree_centrality))

In [ ]:
imn_annotated_stats(path, file_name_in, nodes_list, edge_list)

In [ ]:
\begin{table}[t]
    \centering
    \begin{tabular}{|c|c|c|}
    \hline
    \textbf{Measure} & \textbf{Inter-regional Area} & \textbf{Urban Area} \\  
    \hline
    Nodes &  $\pm$  & 5.06 $\pm$ 1.02 \\ 
    Edges &  $\pm$  & 13.31 $\pm$ 4.56 \\ 
    Density &  $\pm$  & 1.28 $\pm$ 0.32 \\ 
    Degree &  $\pm$  & 5.10 $\pm$ 1.10 \\
    Clus. Coef. &  $\pm$  & 0.85 $\pm$ 0.20 \\
    Degree Centrality &  $\pm$ 0.31 & 1.29 $\pm$ 0.29 \\
    \hline
    \end{tabular}
\end{table}
